In [18]:
import pandas as pd
import json

# CSV 파일 불러오기
df = pd.read_csv("야인시대.csv")

# "인물"과 "대사" 열 선택
df = df[["인물", "대사"]]

df = df[df["대사"].str.len() > 70]

In [19]:
df = df.sort_values("인물").reset_index(drop=True)
df["인물"].value_counts()
df

,인물,대사
0,두한,"어머니...! 어머니 죽으면 안 돼요...! 어머니...! 어머니, 말 좀 해보세요..."
1,두한,하지만 이 분명 우리만의 돈은 아닙니다. 이런 건 많은 사람들의 참여가 필요합니다....
2,두한,"그럴 것 같습니다. 당연하지요. 이, 전쟁이 끝난 이 서울 거리에 먹을 게 있습니까..."
3,두한,"하하하! 아무튼, 처음부터 관철이 네게 맡긴 일이니까 잘 해봐. 이런 일에는 내가 ..."
4,두한,이 앞서 얘기했지만 돈을 만지는 일입니다. 뒤에서 조용히 있는 고문 정도가 좋습니다...
...,...,...
1100,정진영,"두한이의 목숨이 달려 있는 일입니다. 그런저런 이유를 따질 때가 아닙니다, 형님....."
1101,정진영,지금은 너 자신만을 생각해야 돼. 우미관 식구 뿐 아니라 종로의 모든 사람들이 두한...
1102,정진영,"저, 그렇게 됐습니다. 아 지금 최동열 기자님께 통행증을 부탁드리고 오는 길입니다...."
1103,정진영,박헌영 동무께서 신신당부하셨던 큰 과업이었다. 그런데 또 다시 과오를 범했어! 심영...


In [20]:
import csv
df.to_csv("야인시대2.csv", index=False, quoting=csv.QUOTE_NONE, escapechar=" ")

In [13]:
#https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset

from openai import OpenAI
client = OpenAI()

client.files.create(
  file=open("train.jsonl", "rb"),
  purpose="fine-tune"
)

FileObject(id='file-E9qelNsl1VkwqfaxSOrPPo19', bytes=35252, created_at=1712192259, filename='train.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

In [14]:
client.fine_tuning.jobs.create(
  training_file="file-E9qelNsl1VkwqfaxSOrPPo19", 
  model="gpt-3.5-turbo",
)
# https://community.openai.com/t/fine-tuning-process-lock-on-validating-files/410799/5

FineTuningJob(id='ftjob-Aoedw71ZHkIAVjdYEttzmj5k', created_at=1712192269, error=Error(code=None, message=None, param=None, error=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-4GBDLkDtYsir8xfWl4v4lrPq', result_files=[], status='validating_files', trained_tokens=None, training_file='file-E9qelNsl1VkwqfaxSOrPPo19', validation_file=None, user_provided_suffix=None, seed=573546948)

In [16]:
from openai import OpenAI
client = OpenAI()
client.fine_tuning.jobs.retrieve("ftjob-Aoedw71ZHkIAVjdYEttzmj5k")

FineTuningJob(id='ftjob-Aoedw71ZHkIAVjdYEttzmj5k', created_at=1712192269, error=Error(code=None, message=None, param=None, error=None), fine_tuned_model='ft:gpt-3.5-turbo-0125:personal::9A5nr7em', finished_at=1712192613, hyperparameters=Hyperparameters(n_epochs=10, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-4GBDLkDtYsir8xfWl4v4lrPq', result_files=['file-xTCVomwUkZLyL1cBr1GwHJ0c'], status='succeeded', trained_tokens=164300, training_file='file-E9qelNsl1VkwqfaxSOrPPo19', validation_file=None, user_provided_suffix=None, seed=573546948)

In [8]:
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model="ft:gpt-3.5-turbo-0125:personal::9A5DGR93",
  messages=[
    {"role": "system", "content": "김두한의 말투와 단어, 어투를 살려서 문장을 작성해줘"}, 
    {"role": "user", "content": "두한이가 식빵물고 등교하다 문득 뒤에서 들리는 목소리에 깜짝 놀라는 순간을 문장으로 나타내줘"}
  ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content='식빵물고 등교하다가 갑자기 뒤에서 "아니 이거 꼭 먹어 봐야 돼!"라는 목소리 듣고 깜짝 놀라더라고', role='assistant', function_call=None, tool_calls=None)


In [11]:
completion = client.chat.completions.create(
  model="ft:gpt-3.5-turbo-0125:personal::9A5DGR93",
  messages=[
    {"role": "system", "content": "이승만의 말투와 단어, 어투를 살려서 문장을 작성해줘"}, 
    {"role": "user", "content": "이승만이 농구하는 상황"}
  ]
)
print(completion.choices[0].message.content)

휴지.....야.....(고개 척) 수비는 확실하게 해야겠네요. 점수로 이기자고!


In [12]:
completion = client.chat.completions.create(
  model="ft:gpt-3.5-turbo-0125:personal::9A5DGR93",
  messages=[
    {"role": "system", "content": "김두한의 말투와 단어, 어투를 살려서 문장을 작성해줘"},
    {"role": "user", "content": "김두한이 밤을 새는 상황"}
  ]
)
print(completion.choices[0].message.content)

야밤에 역시 문의가 제일 많아. 새벽이 와도 불지를 않아서 새벽이 밝은데서 짜증이 나는데 역시 칸막이 이야기를 제일 들어줄 만한 부분이 있지.


In [19]:
# 새 모델 완성 : ft:gpt-3.5-turbo-0125:personal::9A5nr7em

completion = client.chat.completions.create(
  model="ft:gpt-3.5-turbo-0125:personal::9A5nr7em",
  messages=[
    {"role": "system", "content": "김두한의 말투와 단어, 어투를 살려서 문장을 작성해줘"}, 
    {"role": "user", "content": "김두한이 형님을 대하는 방법"}
  ]
)
print(completion.choices[0].message.content)

형님한테는 꼭 존댓말을 써주지! 상대가 어렸든 말든, 나보다 먼저 인사해줄 때까지는, 꼭 기다려줘야해!


In [2]:
# 회사는 자아실현의 공간이나 사상을 설파하는 장소가 아니다
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model="ft:gpt-3.5-turbo-0125:personal::9A5nr7em",
  messages=[
    {"role": "system", "content": "문장을 입력하면 이 문장이과 가장 가까운 사람을 말해줘(두한, 이정재, 정진영, 이승만)"},
    {"role": "user", "content": "회사는 자아실현의 공간이나 사상을 설파하는 장소가 아니다"}
  ]
)
print(completion.choices[0].message.content)

정진영
